In [3]:
#import scrublet as scr
import numpy as np
import pandas as pd
import csv
import scanpy as sc 
import scanpy.external as sce
import phate
import pickle
import matplotlib
from matplotlib import pyplot as plt
import warnings
from scipy.stats import binom
from scipy.stats import multinomial
import seaborn as sns
from scipy.stats import hypergeom
import warnings
warnings.filterwarnings('ignore')
import itertools
import harmonypy as hm
from random import sample
from scipy.stats import linregress

from sgRNA_Filtering_Functions import gen_sg_combos
from sgRNA_Filtering_Functions import cluster_bias
from sgRNA_Filtering_Functions import CB_Filter_NC

from sgRNA_Filtering_Functions import CB_Filter_SG_Stat
from sgRNA_Filtering_Functions import CB_Filter_SG_Stat_ALL

### CM SGRNA Processing (Subset sg cells, create sg dictionary)

In [35]:
#Load sgRNA Matrix
#SG_DF = pd.read_pickle('DAP3_SGRNA_IND_072821.pkl')

In [36]:
#Load CM PHATE Anndata
#CM_Cells = sc.read_h5ad('DAP3_SC_CM_Pseudo_PHATE_Man_080521.h5ad')

### CM SGRNA Processing (Subset sg cells, create sg dictionary)

In [37]:
# single cell dataset already selected for sg cells. just need to subset SG_DF
SG_CM = SG_DF.loc[CM_Cells.obs.index]

In [38]:
# List of all sgRNA in which only 1 guide exists.
Single_Guide_List= ['NC_GAG', 'NC_GFP', 'NC_GFP_1', 'NC_GFP_2', 'NC_HS2', 'MALAT1']

In [39]:
# Load sgRNA Guide Dictionary
PATH = '../../Data/Pickles/'
Dict_File = open(PATH + "Guide_Dictionary.pkl", "rb")
Guide_Dictionary = pickle.load(Dict_File)

In [40]:
# Load sgRNA Cell Dictionary
PATH = '../../Data/Pickles/'
Dict_File = open(PATH + "CM_Large_sgRNA_Dic.pkl", "rb")
SGRNA_CELLS_DIC = pickle.load(Dict_File)

In [41]:
# List of all NC cells
Clean_NC_Cells = []
[Clean_NC_Cells.extend(SGRNA_CELLS_DIC[i]) for i in Guide_Dictionary['NC']]

[None, None, None, None, None]

### Run CM Bias

In [42]:
# First identifies NC which is biased from the rest. 
p_val_subset = 10
for filter_nc in Guide_Dictionary['NC']:
    cur_pval = []
    Updated_NC_Name = [i for i in Guide_Dictionary['NC'] if i not in filter_nc]
    for nc in Updated_NC_Name:
        
        NC_DF = CM_Cells[SGRNA_CELLS_DIC[nc]]
        NC_Total = NC_DF.shape[0]
        
        Subset_NC = [i for i in Clean_NC_Cells if i not in SGRNA_CELLS_DIC[filter_nc]]
        Back_DF = CM_Cells[Subset_NC]
        Back_Total = Back_DF.shape[0]
        
        for cluster in CM_Cells.obs.louvain.cat.categories.tolist():
            Back_Cluster = Back_DF[Back_DF.obs.louvain == cluster].shape[0]
            NC_Cluster = NC_DF[NC_DF.obs.louvain == cluster].shape[0]

            if hypergeom.cdf(NC_Cluster, Back_Total, NC_Total, Back_Cluster) < 0.05:
                cur_pval.append(1)
            if 1 - hypergeom.cdf(NC_Cluster - 1, Back_Total, NC_Total, Back_Cluster) < 0.05:
                cur_pval.append(1)
    if sum(cur_pval) < p_val_subset:
        p_val_subset = sum(cur_pval)
        Filtered_NC = filter_nc
            

In [43]:
# Save NC which is filtered out
PATH = '../../Data/Pickles/'
a_file = open(PATH + "Filtered_CM_NC.pkl", "wb")
pickle.dump(Filtered_NC, a_file)
a_file.close()

In [44]:
# Remove all cells with biased NC sgRNA
Filtered_NC_List = [i for i in Clean_NC_Cells if i not in SGRNA_CELLS_DIC[Filtered_NC]]

In [51]:
### P_Values for all combos for each guide
PATH = '../../Data/Pickles/'
for Query_Target in Guide_Dictionary.keys():
    if Query_Target in Single_Guide_List:
        continue
    tran_matrix = CM_Cells
    t_dict = Guide_Dictionary
    s_dict = SGRNA_CELLS_DIC
    singlets = Single_Guide_List
    control_list = Filtered_NC_List

    Combo_List = []
    Combo_Str = []
    Target = Query_Target
    target_values_dep = []
    target_values_enr = []

    Combo_List, Combo_Str, Total_Combos = gen_sg_combos(t_dict, Target)
    Combo_Dict = dict(zip(Combo_Str, Combo_List))
        
    for guide_set in Total_Combos:
        CELL_LIST = []
        [CELL_LIST.extend(SGRNA_CELLS_DIC[Target_key]) for Target_key in list(guide_set)]
        
        if Target == 'NC':
            nc_controls = []
            for cell in Filtered_NC_List:
                if cell not in CELL_LIST:
                    nc_controls.append(cell)
            control_list = nc_controls
        ## Calculate hypergeo
        clust_values_dep, cluster_list_dep = cluster_bias(target_cells= CELL_LIST, control_cells = control_list, t_matrix=tran_matrix, direction = 'depletion')
        clust_values_enr, cluster_list_enr = cluster_bias(target_cells= CELL_LIST, control_cells = control_list, t_matrix=tran_matrix, direction = 'enrichment')

        target_values_dep.append(clust_values_dep)
        target_values_enr.append(clust_values_enr)
       
    dep_DAP3 = pd.DataFrame(data = target_values_dep, columns  = cluster_list_dep, index= Combo_Str)         
    dep_DAP3.to_csv(PATH + Query_Target + '_080521_Depletion.csv')
    
    enr_DAP3 = pd.DataFrame(data = target_values_enr, columns  = cluster_list_enr, index= Combo_Str)         
    enr_DAP3.to_csv(PATH + Query_Target + '_080521_Enrichment.csv')
    
    a_file = open(PATH + Target + "_combo_dict_080521.pkl", "wb")
    pickle.dump(Combo_Dict, a_file)
    a_file.close()

### ALL SGRNA Processing (Subset sg cells, create sg dictionary)

In [26]:
#SG_DF = pd.read_pickle('/project/GCRB/Hon_lab/s425140/03.Data/07.Cardiomyocyte_Small_Scale/DAP3/05_28_21/PICKLE/DAP3_SGRNA_IND_SC.pkl')

In [27]:
#SC_DF = sc.read_h5ad('/project/GCRB/Hon_lab/s425140/03.Data/07.Cardiomyocyte_Small_Scale/DAP3/07_28_21_Paper/H5AD/DAP3_SC_072821.h5ad')

In [28]:
### single cell dataset already selected for sg cells. just need to subset sgdf
SG_CM = SG_DF.loc[SC_DF.obs.index]
SC_DF = SC_DF[SG_CM.index]

In [29]:
PATH = '../../Data/Pickles/'
Dict_File = open(PATH + "ALL_Large_sgRNA_Dic.pkl", "rb")
SGRNA_CELLS_DIC = pickle.load(Dict_File)

In [30]:
Clean_NC_Cells = []
[Clean_NC_Cells.extend(SGRNA_CELLS_DIC[i]) for i in Guide_Dictionary['NC']]

[None, None, None, None, None]

### Run ALL Bias

In [32]:
p_val_subset = 10
for filter_nc in Guide_Dictionary['NC']:
    cur_pval = []
    Updated_NC_Name = [i for i in Guide_Dictionary['NC'] if i not in filter_nc]
    for nc in Updated_NC_Name:
        
        NC_DF = SC_DF[SGRNA_CELLS_DIC[nc]]
        NC_Total = NC_DF.shape[0]
        
        Subset_NC = [i for i in Clean_NC_Cells if i not in SGRNA_CELLS_DIC[filter_nc]]
        Back_DF = SC_DF[Subset_NC]
        Back_Total = Back_DF.shape[0]
        
        for cluster in SC_DF.obs.louvain.cat.categories.tolist():
            Back_Cluster = Back_DF[Back_DF.obs.louvain == cluster].shape[0]
            NC_Cluster = NC_DF[NC_DF.obs.louvain == cluster].shape[0]

            if hypergeom.cdf(NC_Cluster, Back_Total, NC_Total, Back_Cluster) < 0.05:
                cur_pval.append(1)
            if 1 - hypergeom.cdf(NC_Cluster - 1, Back_Total, NC_Total, Back_Cluster) < 0.05:
                cur_pval.append(1)
    if sum(cur_pval) < p_val_subset:
        p_val_subset = sum(cur_pval)
        Filtered_NC = filter_nc
            

In [33]:
Guide_Dictionary['NC'] = [i for i in New_NC_Name if i not in Filtered_NC]

In [34]:
Filtered_NC_List = [i for i in Clean_NC_Cells if i not in SGRNA_CELLS_DIC[Filtered_NC]]

In [ ]:
### P_Values for all combos for each guide
PATH = '../../Data/Pickles/'
for Query_Target in Guide_Dictionary.keys():
    if Query_Target in Single_Guide_List:
        continue
    tran_matrix = SC_DF
    t_dict = Guide_Dictionary
    s_dict = SGRNA_CELLS_DIC
    singlets = Single_Guide_List
    control_list = Filtered_NC_List

    Combo_List = []
    Combo_Str = []
    Target = Query_Target
    target_values_dep = []
    target_values_enr = []

    Combo_List, Combo_Str, Total_Combos = gen_sg_combos(t_dict, Target)
    Combo_Dict = dict(zip(Combo_Str, Combo_List))
        
    for guide_set in Total_Combos:
        CELL_LIST = []
        [CELL_LIST.extend(SGRNA_CELLS_DIC[Target_key]) for Target_key in list(guide_set)]
        
        if Target == 'NC':
            nc_controls = []
            for cell in Filtered_NC_List:
                if cell not in CELL_LIST:
                    nc_controls.append(cell)
            control_list = nc_controls
        ## Calculate hypergeo
        clust_values_dep, cluster_list_dep = cluster_bias(target_cells= CELL_LIST, control_cells = control_list, t_matrix=tran_matrix, direction = 'depletion')
        clust_values_enr, cluster_list_enr = cluster_bias(target_cells= CELL_LIST, control_cells = control_list, t_matrix=tran_matrix, direction = 'enrichment')
        
        target_values_dep.append(clust_values_dep)
        target_values_enr.append(clust_values_enr)
       
    dep_DAP3 = pd.DataFrame(data = target_values_dep, columns  = cluster_list_dep, index= Combo_Str)         
    dep_DAP3.to_csv(PATH + Query_Target + '_080521_Depletion_ALL.csv')
    
    enr_DAP3 = pd.DataFrame(data = target_values_enr, columns  = cluster_list_enr, index= Combo_Str)         
    enr_DAP3.to_csv(PATH + Query_Target + '_080521_Enrichment_ALL.csv')
    
    a_file = open(PATH + Target + "_combo_dict_080521_ALL.pkl", "wb")
    pickle.dump(Combo_Dict, a_file)
    a_file.close()        